In [3]:
import re, html, pandas as pd, numpy as np, json
from pathlib import Path
from collections import defaultdict
import plotly.graph_objects as go
import pycountry

# -------------------- Load and parse --------------------
txt_path = Path("legacy_HTML.txt")
raw = txt_path.read_text(encoding="utf-8")
html_text = html.unescape(raw)

cat_re = re.compile(r"<p><b>([^<]+)</b></p>", re.IGNORECASE)
anchor_re = re.compile(r'(<a href="">([^<]+)</a>)', re.IGNORECASE)

# Linkify plain URLs so they’re clickable
url_re = re.compile(r'(?<!href=[\'"])(https?://[^\s<>"\'\)\]]+)')
def linkify_urls(s: str) -> str:
    return url_re.sub(lambda m: f'<a href="{m.group(1)}" target="_blank" rel="noopener">{m.group(1)}</a>', s or "")

# Find all category blocks
cat_spans = [(m.group(1).strip(), m.start(), m.end()) for m in cat_re.finditer(html_text)]
country_blocks = defaultdict(list)
for i, (cat, s, e) in enumerate(cat_spans):
    seg_start = e
    seg_end = cat_spans[i+1][1] if i+1 < len(cat_spans) else len(html_text)
    segment = html_text[seg_start:seg_end]
    anchors = list(anchor_re.finditer(segment))
    for j, am in enumerate(anchors):
        name = am.group(2).strip()
        if not name or name.lower() == "#world":
            continue
        text_start = am.end()
        text_end = anchors[j+1].start() if j+1 < len(anchors) else len(segment)
        snippet_html = (segment[text_start:text_end].strip() or "<p>No additional details found.</p>")
        snippet_html = linkify_urls(snippet_html)  # <-- make any raw URLs clickable
        country_blocks[cat].append((name, snippet_html))

# -------------------- Normalize names, capture EU snippet --------------------
EXCLUDE_FROM_MAP = {"European Union"}

alias = {
    "The Bahamas": "Bahamas",
    "The Gambia": "Gambia",
    "Ivory Coast": "Côte d'Ivoire",
    "Republic of the Congo": "Congo",
    "Democratic Republic of Congo": "Democratic Republic of the Congo",
    "Czech Republic": "Czechia",
    "Macedonia": "North Macedonia",
    "Kosovo": "Kosovo",
    "South Korea": "South Korea",
    "North Korea": "North Korea",
    "Syria": "Syria",
    "Palestine": "Palestine",
}

def resolve_name(name):
    name = alias.get(name, name)
    try:
        return pycountry.countries.lookup(name).name
    except Exception:
        return name

eu_snippet = ""
records = []
for cat, items in country_blocks.items():
    for country, snippet in items:
        if country == "European Union":
            eu_snippet = linkify_urls(snippet)  # ensure EU snippet links are clickable
            continue
        resolved = resolve_name(country)
        records.append({
            "raw_name": country,
            "name": resolved,
            "category": cat,
            "html": snippet
        })

df = pd.DataFrame(records)

# -------------------- EU rule --------------------
eu_members = {
    "Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia","Finland",
    "France","Germany","Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg",
    "Malta","Netherlands","Poland","Portugal","Romania","Slovakia","Slovenia","Spain","Sweden"
}

df["name"] = df["name"].fillna(df["raw_name"])
df.loc[df["name"].isin(eu_members), "category"] = "Import substitution"

missing = sorted(eu_members - set(df["name"]))
if missing:
    df = pd.concat(
        [df, pd.DataFrame([{
            "raw_name": m,
            "name": m,
            "category": "Import substitution",
            "html": eu_snippet or "<p>European Union measures applicable.</p>"
        } for m in missing])],
        ignore_index=True
    )

if eu_snippet:
    df.loc[df["name"].isin(eu_members), "html"] = eu_snippet

df = df.drop_duplicates(subset=["name"], keep="first")

# -------------------- Plotly country name normalization --------------------
plotly_alias = {
    "Taiwan, Province of China": "Taiwan",
    "Côte d'Ivoire": "Ivory Coast",
    "Syrian Arab Republic": "Syria",
    "Lao People's Democratic Republic": "Laos",
    "Iran, Islamic Republic of": "Iran",
    "Korea, Republic of": "South Korea",
    "Korea, Democratic People's Republic of": "North Korea",
    "Moldova, Republic of": "Moldova",
    "Tanzania, United Republic of": "Tanzania",
    "Venezuela, Bolivarian Republic of": "Venezuela",
    "Bolivia, Plurinational State of": "Bolivia",
    "Russian Federation": "Russia",
    "Congo": "Republic of the Congo",
    "Congo, The Democratic Republic of": "Democratic Republic of the Congo",
    "Eswatini": "Eswatini",
    "Cabo Verde": "Cape Verde",
}
def to_plotly_name(name): return plotly_alias.get(name, name)
df["plotly_name"] = df["name"].apply(to_plotly_name)

# -------------------- Categories & colors --------------------
categories_order = [
    "Isolation","Import substitution","Bank paralysis","Personal sanctions",
    "Export and financial controls","Minor sanctionary measures",
    "Condemnation without action","Ambiguous stance",
    "Leaning towards Russia","Partnership with Russia","Obedience to Russia",
]
palette = [
    "maroon","red","#FE8116","#FFA80F","#FFCF07",
    "#FFF600","#FFFEE9","#EBE8FC","#9f55c5","#7b3fae","#52307c",
]
color_map = dict(zip(categories_order, palette))

# -------------------- Figure --------------------
fig = go.Figure()
for idx, cat in enumerate(categories_order):
    sub = df[df["category"] == cat]
    if sub.empty: continue
    fig.add_trace(go.Choropleth(
        locations=sub["plotly_name"],
        locationmode="country names",
        z=[idx]*len(sub),
        text=sub["raw_name"] + " — " + sub["category"],
        hovertemplate="<b>%{location}</b><br>%{text}<extra></extra>",
        colorscale=[[0, color_map[cat]], [1, color_map[cat]]],
        showscale=False,
        marker_line_color="#d3d3d3",
        marker_line_width=0.5,
        name=cat,
        hoverlabel=dict(namelength=-1),
        customdata=np.stack([sub["plotly_name"], sub["raw_name"]], axis=1)
    ))

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True, projection_type="natural earth", bgcolor="rgba(0,0,0,0)"),
    margin=dict(l=0,r=0,t=50,b=0),
    paper_bgcolor="white"
)

# -------------------- Legend HTML --------------------
legend_items = "".join(
    f'<div style="display:flex;align-items:center;margin:4px 12px 4px 0;">'
    f'<span style="display:inline-block;width:14px;height:14px;background:{color_map[c]};border-radius:50%;margin-right:8px;border:1px solid #999;"></span>'
    f'<span style="font:14px/1.2 system-ui,-apple-system,Segoe UI,Roboto,Helvetica,Arial,sans-serif;">{c}</span>'
    f'</div>'
    for c in categories_order
)

# -------------------- Country -> HTML snippet mapping --------------------
country_html = {row["plotly_name"]: (row.get("html") or "") for _, row in df.iterrows()}

# -------------------- Build HTML --------------------
def to_serializable(o):
    if isinstance(o, np.ndarray): return o.tolist()
    if isinstance(o, dict): return {k: to_serializable(v) for k,v in o.items()}
    if isinstance(o, (list, tuple)): return [to_serializable(x) for x in o]
    return o

fig_dict = to_serializable(fig.to_dict())

panel_css = """
.wrap{max-width:1280px;margin:0 auto;padding:16px}
#map{width:100%;height:720px;border:1px solid #eee}
.legend{display:flex;flex-wrap:wrap;margin-top:8px}
#panel{
  position: fixed; top: 0; right: -520px; width: 520px; height: 100vh;
  background: #fff; box-shadow: -8px 0 24px rgba(0,0,0,.08);
  border-left: 1px solid #eee; padding: 20px 20px 28px; overflow:auto;
  transition: right .35s ease; z-index: 1000;
}
#panel.open{ right: 0; }
#panel h3{ margin: 0 0 8px; font: 600 20px/1.25 system-ui,-apple-system,Segoe UI,Roboto,Helvetica,Arial,sans-serif;}
#panel .chip{display:inline-block;font:12px/1.6 system-ui;background:#e0e0e0;border:1px solid #b0b0b0;color:#333;border-radius:999px;padding:2px 10px;margin-bottom:10px}
#panel .content p{ margin:.5em 0; font:14px/1.5 system-ui,-apple-system,Segoe UI,Roboto,Helvetica,Arial,sans-serif;}
#panel .content a{ color:#1a73e8; text-decoration:underline; word-break:break-word;}
#panel .content a:visited{ color:#1a73e8; }
#panel .close{ position:absolute; top:10px; right:12px; border:none; background:transparent; font-size:22px; cursor:pointer;}
"""

html_out = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Sanctions World Map — clickable</title>
<script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>
<style>body{{margin:0;padding:0;background:#fff}} {panel_css}</style>
</head>
<body>
<div class="wrap">
  <h2 style="font:600 20px/1.3 system-ui;margin:0 0 4px;">World response to Russia: sanction stance by country</h2>
  <div style="font:14px/1.3 system-ui;color:#555;margin:0 0 10px;">Last updated March 2022</div>
  <div id="map"></div>
  <div class="legend">{legend_items}</div>
</div>

<div id="panel">
  <button class="close" aria-label="Close" onclick="document.getElementById('panel').classList.remove('open')">×</button>
  <h3 id="panel-title"></h3>
  <div class="chip" id="panel-category"></div>
  <div class="content" id="panel-content"></div>
</div>

<script>
var figure = {json.dumps(fig_dict)};
var countryHtml = {json.dumps(country_html)};

// Render map and handle clicks
Plotly.newPlot("map", figure.data, figure.layout, {{responsive: true}}).then(function(gd){{
  gd.on('plotly_click', function(evt){{
    if(!evt || !evt.points || !evt.points.length) return;
    var p = evt.points[0];
    var country = p.location;
    var traceName = (gd.data[p.curveNumber] || {{}}).name || '';
    var html = countryHtml[country] || "<p>No details found.</p>";
    document.getElementById('panel-title').textContent = country;
    var chip = document.getElementById('panel-category');
    chip.textContent = traceName;
    // Chip stays grey/dark via CSS (always-on)
    document.getElementById('panel-content').innerHTML = html;
    document.getElementById('panel').classList.add('open');
  }});
}});
</script>
</body>
</html>
"""

out_path = "index.html"
Path(out_path).write_text(html_out, encoding="utf-8")
print("Wrote:", out_path)

Wrote: index.html
